In [ ]:
import os
import cv2
import numpy as np 
import tensorflow as tf
import filetype
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random
import ssl
import datetime
import sys
import copy
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from skimage.io import imread, imsave
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import preprocess_input
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
from keras import backend as K
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

In [ ]:
#調整亮度與銳利度
def automatic_brightness_and_contrast(image, clip_hist_percent=25):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha

    auto_result = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return (auto_result, alpha, beta)

In [ ]:
def read_directory(directory_name,data,label,label_name,data_position):
    global count_image
    count_image_before = copy.deepcopy(count_image)
    for filename in os.listdir(directory_name):
      if filename == '.DS_Store':
          continue
      count_image+=1
      img = tf.keras.preprocessing.image.load_img(directory_name + "/" + filename,target_size=(256,256,3)) #讀圖片
      img = tf.keras.preprocessing.image.img_to_array(img)
      auto_result, alpha, beta = automatic_brightness_and_contrast(img) #亮度與銳利度調整
      img = tf.keras.preprocessing.image.img_to_array(auto_result)
      img = tf.keras.applications.densenet.preprocess_input(img) #densenet的顏色前處理
      data.append(img)
      label.append(label_name)
    data_position.append((count_image_before,count_image))
    print(count_image)

In [ ]:
train_dir = '' #輸入自己的資料夾路徑 形式：train/A/, train/B/, train/C/
test_dir = '' #輸入自己的資料夾路徑 形式：test/A/, test/B/, test/C/

train_data = [] 
train_label = []

test_data = []
test_label = []

def load_data(img_dir,data,label,label_number,data_position):
    for count,filename in enumerate(os.listdir(img_dir)):
        if filename == '.DS_Store':
                continue
        if os.path.isfile(img_dir + '/' + filename):
            print ("it's a normal file")
        else:
            print(filename+" it's a folder")
            for folder in os.listdir(img_dir+'/'+filename):
                if folder == '.DS_Store':
                    continue 
                label_number+= 1
                print(img_dir+'/'+filename+"/"+folder + " : "+str(label_number))
                read_directory(img_dir+'/'+filename+"/"+folder,data,label,label_number,data_position)
    return label_number  

In [ ]:
#記得label是從1開始 位置是從0

count_folder_train = 0 #計算train有幾個資料夾
count_image = 0  #計算多少張照片
train_data_position = [] #train每一個類別的位置
count_folder_train = load_data(train_dir,train_data,train_label,count_folder_train,train_data_position)
print(count_folder_train)
count_image_train = copy.deepcopy(count_image) #train總共有幾張

count_folder_test = 0 #test有幾個資料夾
count_image = 0 
test_data_position = [] #test每一個類別的位置
count_folder_test = load_data(test_dir,test_data,test_label,count_folder_test,test_data_position)
print(count_folder_test)
count_image_test  = copy.deepcopy(count_image) #test總共有多少張

In [ ]:
train_data, train_label = shuffle(train_data, train_label, random_state=0) #把資料打散,這樣取圖片的時候可以更隨機

In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)
test_data = np.array(test_data)
test_label = np.array(test_label)

In [ ]:
#看資料使否轉換正常
train_data.shape,test_data.shape,train_label.shape, test_label.shape

In [ ]:
#建立model
inc = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
    pooling='max',
)
def create_model():
    model = tf.keras.models.Sequential([
        inc,
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(512,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation='sigmoid')    
    ])
    return model
    
embedding_model = create_model()
inc.summary(),embedding_model.summary()

In [ ]:
# 設定net的input樣式
in_anc = tf.keras.layers.Input(shape=(256,256,3))
in_pos = tf.keras.layers.Input(shape=(256,256,3))
in_neg = tf.keras.layers.Input(shape=(256,256,3))

#樣式輸入
em_anc = embedding_model(in_anc)
em_pos = embedding_model(in_pos)
em_neg = embedding_model(in_neg)

#把輸入連接起來
out = tf.keras.layers.concatenate([em_anc,em_pos,em_neg],axis=1)

net = tf.keras.models.Model(
    [in_anc,in_pos,in_neg],
    out
)
net.summary()

In [ ]:
def create_batch_group(group_data,group_label,batch_size):
    #建立目標數量大小的list
    anchors = np.zeros((batch_size,256,256,3))
    positives = np.zeros((batch_size,256,256,3))
    negatives = np.zeros((batch_size,256,256,3))

    #開始建立batch
    for i in range(0,batch_size):
        index = random.randint(0,(len(group_data)-1)) #隨機取一個圖片的位置
        anc = group_data[index] #得到該圖片
        y = group_label[index] #得到該圖片的數字

        indices_for_pos = np.squeeze(np.where(group_label == y)) #所有與該圖片相同數字的位置
        indices_for_neg = np.squeeze(np.where(group_label != y)) #所有與該圖片不相同數字的位置
        
        #上述位置隨機取一個圖片
        pos = group_data[indices_for_pos[random.randint(0,len(indices_for_pos)-1)]] 
        neg = group_data[indices_for_neg[random.randint(0,len(indices_for_neg)-1)]]

        anchors[i] = anc
        positives[i] = pos
        negatives[i] = neg

    #回傳一個3列 size行的陣列
    return [anchors,positives,negatives]

In [ ]:
#測試是否生出三張圖片
temp = create_batch_group(train_data,train_label,1)
plt.subplot(1,3,1)
temp1 = tf.keras.preprocessing.image.array_to_img(temp[0][0])
plt.imshow(temp1)
plt.xticks([])
plt.yticks([])

plt.subplot(1,3,2)
temp1 = tf.keras.preprocessing.image.array_to_img(temp[1][0])
plt.imshow(temp1)
plt.xticks([])
plt.yticks([])

plt.subplot(1,3,3)
temp1 = tf.keras.preprocessing.image.array_to_img(temp[2][0])
plt.imshow(temp1)
plt.xticks([])
plt.yticks([])

In [ ]:
def triplet_loss(alpha, emb_dim):
    def loss(y_true,y_pred):
        anc, pos, neg = y_pred[:, :emb_dim], y_pred[:, emb_dim:2*emb_dim], y_pred[:, 2*emb_dim:]
        dp = tf.reduce_mean(tf.square(anc-pos), axis=1)
        dn = tf.reduce_mean(tf.square(anc-neg), axis=1)
        return tf.maximum(dp - dn +alpha,0.)
    return loss
    
def data_generator(group_data, group_label, batch_size, emb_dim):
    while True:
      x = create_batch_group(group_data,group_label,batch_size)
      y = np.zeros((batch_size, 3*emb_dim))
      yield x, y 

In [ ]:
net.compile(loss = triplet_loss(alpha=0.2, emb_dim=64),optimizer='adam')
LR_function = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=1) #學習率調整
history = net.fit(
    data_generator(train_data,train_label,10,64),
    epochs= 10, steps_per_epoch= len(train_data)//10,
    validation_data = data_generator(train_data,train_label,10,64),
    validation_steps = len(train_data)//10,
    callbacks=[LR_function],
    verbose = True)